##### ---
# Unit05: Processing passive seismic data III

This notebook has some practical processing activities of the Course **ProSeisSN**. It deals with event detection using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***
#### The code $\Downarrow$ BELOW $\Downarrow$ runs a notebook with other dependencies

In [ ]:
#------ Import Libraries
import sys
import os
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)

#------ Magic commands
%matplotlib inline
%matplotlib widget

---
## Read data locally

Read the demeaned+detrended Z-component: ../Data/wb11793085d.mseed

|EventID | Time | Lat | Long | Depth(km) | Mww | Location |
| :- | :-: | :- | :- | :-: | :-: | :- |
|11793085|2024-01-20T21:31|-7.2879|-71.464|607|6.6|WESTERN BRAZIL|

recorded at station IU HKT, $44^{\circ}$ away ($\gtrsim4440$km).

### The relevant limits in time and frequency domains
- **time-domain**

$$\underset{0}{\mid}\cdot\cdotp\cdotp\cdotp\cdot\cdotp\cdotp\cdotp\underset{\delta t}{\mid\mid}\cdot\cdotp\cdotp\cdotp\cdot\cdotp\cdotp\cdotp\underset{T_{w}}{\!\cdotp\!\mid N}$$

- **frequency-domain**

$$\underset{0}{\mid}\cdotp\!\underset{f_{min}}{\cdotp\cdotp\mid\cdotp\cdotp}
\cdot\cdotp\cdotp\cdotp\cdot\cdotp\cdotp\cdotp\underset{\delta f}{\mid\mid}\cdot\cdotp\cdotp\cdotp\cdot\underset{3 f_{c}}{\cdotp\cdotp\mid\cdotp\cdotp}\cdotp\cdotp\cdotp\underset{\zeta f_{Ny}}{\cdotp\cdotp\mid\cdotp\cdotp}\cdotp\underset{f_{Ny}}{\!\cdotp\!\mid}$$

In [ ]:
#%%script echo skipping
"""
=========== READ THE DATA LOCALLY ===========
"""
filename = '../Data/wb11793085d.mseed'
print(f"Read the demean+detrend data {filename}")
st = read(filename)
#------- Begin recording a new processing flux
Pflux = ['Unit05 '+filename]

#\________________________________________________________________________________________________/
#------- Sampling frequency
print(f">> The sampling frequency is {np.round(st[0].stats.sampling_rate, 5)}Hz")
print(f">> The original Nyquist is {np.round(1. / (2.0 * st[0].stats.delta), 5)}Hz")
#--- relative time: nummpy array
time = st[0].times(type="relative")
print(f">> The time window is tw = {time[-1]}s, with npts = {st[0].stats.npts} samples")

#-------- Spectrogram
p.Pspect(time, st[0])

#------- Deep copy of the Z-component
trZ = st[0].copy()
tr0 = st[0].copy()

---
## Filter the data 
- The **P-wave** disappear for *HP*, fc>=0.5!
- Hint: bp .1  4.

In [ ]:
#------- Filter data
trZ, ftype, flims = u.TrFlt(trZ)
#           +---+─> [fmin, fmax] = useful frequency range due to filter
# Unpack useful frequency range
fmin, fmax = flims

print(flims)
#-------- Data spectrogram
p.Pspect(time, trZ)

#--------  Save trace for the next cell + append to the processing flux
tr0, trZ, Pflux = u.AuxReset(tr0, trZ, lst=Pflux, app=[ftype])

---
## Resampling the data
- Change the sampling rate of the data.
- **It is necessary to lowpass filter the data with a new Nyquist frequency before downsampling.**

In [ ]:
#------- Decide on resampling
dummy = 1. / tr0.stats.delta
print(f"\n>> The data have {tr0.stats.npts} samples, with sampling rate of {dummy}Hz")
ent = input(f' Enter a new sampling rate <{dummy / 2.}Hz (dflt: no resampling) in Hz: ') or '-1'
ent = float(ent)

#------- Resampling
if ent > 0. and ent < dummy / 2.:
#--- Apply an acausal filter at corner frequency fc, zero phase and resample,
#     if lower than filter above.
    if ent < fmax: trZ.filter('lowpass', freq=ent, zerophase=True, corners=4)
#--- Resample
    trZ.resample(sampling_rate=ent)
    dummy = 1. / trZ.stats.delta
    print(f"\n>> The data now have {trZ.stats.npts} samples, with a sampling rate of {dummy}s")
    
#-------- Spectrogram
    time = trZ.times(type="relative")
    p.Pspect(time, trZ)    
    
#--------  Save trace to the next cell + append to the processing flux
    dummy = 'decimation '+str(ent)
    tr0, trZ, Pflux = u.AuxReset(tr0, trZ, lst=Pflux, app=[dummy])

else:
    dummy = 1. / trZ.stats.delta
    print(f">> The original sampling rate of {dummy}Hz is unchanged.")

## Taper the data window
- Use the **Hanning Window**, which uses a cosine function to taper at both ends. Hint: 0.2.

In [ ]:
#------- Taper the data
ent = input(f' Enter the percentage to taper window ends (rtn=0.2)\n') or '0.2'
ent = float(ent)
# Apply Hanning on the signal
trZ.taper(type = 'hann', max_percentage = ent)

#--------  Save trace to the next cell + append to the processing flux
    dummy = 'Hanning '+str(ent)
    tr0, trZ, Pflux = u.AuxReset(tr0, trZ, lst=Pflux, app=[dummy])

---
## Estimating the the instantaneous value of a seismic signal
The instantaneous absolute amplitude of a signal is estimated using its envelope, or the modulus of the analytic signal. The analytic of a *real* signal $s\left(t\right)$ is 

$$
\mathcal{S}_A\left(t\right)\:=\:s\left(t\right)+\imath\mathcal{H}\left(s\left(t\right)\right)\:=\:s\left(t\right)\exp\left(\imath\mathcal{H}\left(s\left(t\right)\right)\right)\:=\:s\left(t\right)\exp\left(\imath\theta\right),
$$
a complex signal, where
$$
\mathcal{H}\left(s\left(t\right)\right)\:=\:s\left(t\right)\star\frac{1}{\pi t}
$$
is its Hilbert transform, where $\star$ denotes a convolution.

The Hilbert transform does not change the magnitude of the Fourier transform of the signal, only its phase:
1) **positive** frequencies are multiplied by $-\imath$ $\rightarrow\varDelta\theta=-\frac{\pi}{2}$;
2) **negative** frequencies are multiplied by $\:\imath$ $\rightarrow\varDelta\theta=\:\frac{\pi}{2}$.

In [ ]:
from scipy.signal import hilbert

#import obspy.signal.filter
#trE = np.abs(obspy.signal.filter.hilbert(trZ.data))

#------- Calculate the envelope of filtered data using the Hilbert transform
trE = np.abs( hilbert(trZ.data) )

#------- Plot trace and envelope
plt.figure(figsize=(10, 6))
plt.plot(time, trZ.data, 'b-', label="Seismic Trace")
plt.plot(time, trE, 'r-', label="Envelope")
#plt.plot(time[peaks], trE[peaks], 'r-', label="Envelope")
plt.xlabel("(s)")
plt.ylabel("Amplitude")
#plt.title(" ")
plt.legend()
plt.grid(True)
plt.show()

---

## Phase detection

### The STA/LTA trigger
The short-time-average/long-time-average **STA/LTA** trigger is usually used in weak-motion applications. The seismic signals are filtered in two moving time windows:
- a short-time average window **STA**, which can estimate an *instantaneous* amplitude of the signal, and
- a long-time average window **LTA**, which estimates a current average noise amplitude.

The STA window measures the *instant* value of a seismic signal and noise, their **envelope**.

Estimate the average of absolute amplitudes in both windows $\Rightarrow$ get the **STA/LTA** ratio $\Rightarrow$  compare with a threshold level. Seismic networks and recorders have a built in **trigger voting** mechanism: how many and which channels have to be in a triggered state before an event is recorded. For simplicity let's use the vertical component of the Acre event.

<img src="StaLta1.png" width="500">

---
### Using the STA/LTA trigger algorithm
- The STA window length $\rightarrow$ $\Delta T_{STA}$: measures the average amplitude of the seismic **signals**.
1) **longer** than a few periods of a typically expected signal.
2) **shorter** than the shortest events we expect to detect.
3) **shorter window** $\rightarrow$ **higher** sensitivity to **short duration/local** earthquakes.
4) **longer window**  $\rightarrow$ **lower**  sensitivity to short duration/local earthquakes $\rightarrow$ focous on **longer duration/distant** earthquakes.
5) **local**    events $\rightarrow$ $\Delta T_{STA}=\left[0.3,0.5\right]s$.
6) **regional** events $\rightarrow$ $\Delta T_{STA}=\left[1,2\right]s$.

- The LTA window length $\Delta T_{LTA}$, measures the average amplitude of the seismic **noise**, with lengths **larger** than a few periods of noise fluctuations.
1) **Shorter** $\Delta T_{LTA}$ can be less sensitive to *regional* events or to a *noisy* site.
2) **Larger** lengths can be more sensitive to distant *regional* events with longer S-P times and conspicuous P waves. Some adjustements on $\Delta T_{LTA}$ can turn the detection more or less sensitive to regional events within epicentral distances in the range of [200-1500]km, which have relatively low-amplitude onsets of P*n* waves.
3) A value of $\Delta T_{LTA}=1$s can be used as an initial value. 

- The relative occurences of **positive**, **false positive** and **missed positive** is conditioned by the STA/LTA trigger threshold level $\mathcal{T}_{STA/LTA}$.
1) Average quiet seismic site/weak events $\rightarrow\,\mathcal{T}_{STA/LTA}\approx4$.
2) Strong-motion/noisier sites $\rightarrow\,\mathcal{T}_{STA/LTA}\gtrsim8$.

|$\mathcal{T}_{STA/LTA}\rightarrow0$|$\mathcal{T}_{STA/LTA}\rightarrow\infty$ |
| :-: | :-: |
|more events and false-triggers | more misses + fewer false-triggers|
|stationary seismic noise | irregular seismic noise levels|

- The termination of data recording is estimated with the STA/LTA detrigger threshold level $\widetilde{\mathcal{T}}_{STA/LTA}$.
1) Average quiet seismic site/weak events $\rightarrow\,\widetilde{\mathcal{T}}_{STA/LTA}\approx2-3$.
2) Strong-motion/noisier sites $\rightarrow\,\widetilde{\mathcal{T}}_{STA/LTA}>3$.


|$\widetilde{\mathcal{T}}_{STA/LTA}\rightarrow0$|$\widetilde{\mathcal{T}}_{STA/LTA}\rightarrow\infty$ |
| :- | :- |
|more coda waves, long records | less coda waves, shorter records|

<img src="StaLta2.png" width="500">

#### STA/LTA Methods
| Method| Description |
| :-: | :-: |
| classic_sta_lta(a, nsta, nita) | Standard STA/LTA |
| recursive_sta_lta(a, nsta, nlta) | Recursive STA/LTA |
| carl_sta_trig(a, nsta, nita, ratio, quiet) | Characteristic function |
| delayed_sta_lta(a, nsta, nlta) | A delayed STA/LTA |
| z_detect(a, nsta) | A Z-detector |

Other Methods:
- pk_baer (reltrc, samp_int, tdownmax, ...[, ...]), a wrapper for P-picker routine by M
- ar_pick {a, b, c, samp_rate, f1, f2, Ita_p, ...), pick P and S arrivals with an AR-AIC + STA/LTA algorithm

In [ ]:
#------ Import Packages
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import z_detect
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import carl_sta_trig
from obspy.signal.trigger import delayed_sta_lta

In [ ]:
"""
Parameters:
nsta(int) – Length of short time average window in samples
nlta(int) – Length of long time average window in samples
"""
print(f">> Sampling interval = {np.round(trZ.stats.delta, 5)}s")
print(f">> Sampling rate = {np.round(trZ.stats.sampling_rate, 5)}Hz")

#------ Enter STA and LTA in s.
ent = input(f'<< Enter STA and LTA in s: ')
ent = ent.rstrip().split(' ')
sta = float(ent[0])
lta = float(ent[1])
#
dummy = sta
sta = int(sta * trZ.stats.sampling_rate)
print(f">> sta = {dummy}s or {sta} data points")
#
dummy = lta
lta = int(lta * trZ.stats.sampling_rate)
print(f">> lta = {dummy}s or {lta} data points")
#
#------ Enter trigger and detrigger thresholds.
ent = input(f'<< Enter trigger and detrigger thresholds: ')
ent = ent.rstrip().split(' ')
trg  = float(ent[0])
dtrg = float(ent[1])

In [ ]:
#------ Classic Sta/Lta
#------ STA and LTA hints: 5 20
#------ Trigger and detrigger hints: 1.5 0.5
#
cft = classic_sta_lta(trZ.data, sta, lta)
#
plot_trigger(trZ, cft, trg, dtrg)

In [ ]:
#------ Z-Detect Sta/Lta
#------ LTA hint: 10
#------ Trigger and detrigger hints: -0.4 -0.3
#
cft = z_detect(trZ.data, lta)
#
plot_trigger(trZ, cft, trg, dtrg)

In [ ]:
#------ Delayed Sta/Lta
#------ LTA hint: 5 10
#------ Trigger and detrigger hints: 1.5 12
cft = delayed_sta_lta(trZ.data, sta, lta)
#
plot_trigger(trZ, cft, trg, dtrg)

In [ ]:
#------ Characteristic function
#------ LTA hint: 5 10
#------ Trigger and detrigger hints: 20.0 -20.0
#
cft = carl_sta_trig(trZ.data, sta, lta, 0.8, 0.8)
#
plot_trigger(trZ, cft, trg, dtrg)

In [ ]:
#------ Recursive Sta/Lta
#------ STA and LTA hints: 5 10
#------ Trigger and detrigger hints: 1.5 0.5
cft = recursive_sta_lta(trZ.data, sta, lta)
#
plot_trigger(trZ, cft, trg, dtrg)

---
